In [ ]:
from azure.data.tables import TableServiceClient

In [ ]:
ConStr = r"DefaultEndpointsProtocol=https;AccountName=gasfundiesdeltalake;AccountKey=CnR2PDBBci1fx26qnA9N2i3QfAeWkwuI4qg3Fv96oJRIbNGFvNX8biuyFcyPCiD2jhwYBWkp4Xac+AStBxnY7A==;EndpointSuffix=core.windows.net"

In [ ]:
type(tmp)

In [ ]:
from contextlib import contextmanager

@contextmanager
def getTable(tableName: str):
    try:
        with TableServiceClient.from_connection_string(conn_str=ConStr) as client:
            yield client.create_table_if_not_exists(table_name=tableName)
    finally:
        pass

In [ ]:
entity1 = {"PartitionKey": "pk001", "RowKey": "rk001", "Value": 4, "day": "Monday", "float": 4.003}
entity2 = {"PartitionKey": "pk001", "RowKey": "rk002", "Value": 4, "day": "Tuesday", "float": 4.003}
entity3 = {"PartitionKey": "pk001", "RowKey": "rk003", "Value": 4, "day": "Wednesday", "float": 4.003}
entity4 = {"PartitionKey": "pk001", "RowKey": "rk004", "Value": 4, "day": "Thursday", "float": 4.003}

# Instantiate a TableClient using a connection string
with getTable('SomeRandomTable') as table_client:

    operations = [
        ("create", entity1),
        # ("delete", entity2),
        ("upsert", entity3),
        # ("update", entity4, {"mode": "replace"}),
    ]
    try:
        table_client.submit_transaction(operations)
    except Exception as e:
        print("There was an error with the transaction operation")
        print(f"Error: {e}")
    # [END batching]

In [ ]:
import pandas as pd

In [ ]:
df1= pd.read_csv('src/Models/PipelineMetaDatahch.csv')
df1.columns

In [ ]:
import duckdb


In [ ]:
df1

In [ ]:
with duckdb.connect('src/Models/EnbridgeMeta.db') as con:
    con.execute("""
        CREATE TABLE IF NOT EXISTS EnbridgeSegmentsTable AS SELECT * FROM df1;
    """)

In [ ]:
df2= pd.read_csv('src/Models/MetaData.csv')
df2.columns

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df1['LocZone_Del'].unique()

In [ ]:
df2['Loc Zone'].unique()

In [ ]:
{
    'EffDate' : 'Eff Date',
    'DirFlow' : 'Dir Flow',
    'Loc' : 'Loc',
    'LocCnty' : 'Loc Cnty',
    'LocName' : 'Loc Name',
    'LocStAbbrev' : 'Loc St Abbrev',
    'LocTypeInd' : 'Loc Type Ind',
    'LocZone_Del' : '',
    
    
    
    'LineNom' : 'Line Nom',
    'LineType' : 'Line Type',
    'PipeGrade' : 'Pipe Grade',
    'SpecCoat' : 'Spec Coat',
    'YearBuilt' : 'Year Built', 
    
}



In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('src/Models/OA_Processed.csv')

In [ ]:
df.info()

In [ ]:
df['TSP'].apply(lambda x: len(str(x))).max()

In [ ]:
df['GFLOC'] = df['GFPipeID'].astype(str)+ df['Loc'].apply(lambda x: x.rjust(6,'0'))

In [ ]:
df['GFLOC']

In [ ]:
df['Loc']

In [1]:
import polars as pl
from pathlib import Path
import functools
import operator

In [2]:
from src.artifacts import getPipes
from src.enbridgescrape import formatOA

In [3]:
Path('./downloads/enbridge/OA').resolve()

PosixPath('/Users/saijagadeeshyadavalli/Coding/Particles/GasFundies/enbridgeGitDump/downloads/enbridge/OA')

In [8]:
pipesLF=pl.LazyFrame(getPipes('Enbridge'))

In [9]:
dfOA = pl.LazyFrame(formatOA())

In [17]:
joinset=set(dfOA.join(pipesLF, on='TSP', how='inner').collect().to_pandas()['TSP'].unique())

In [15]:
OAset = set(formatOA().to_pandas()['TSP'].unique())

In [18]:
OAset - joinset

{np.int64(36638044), np.int64(612195573), np.int64(946822749)}

In [19]:
dfOA = formatOA().to_pandas()

In [21]:
dfOA[dfOA['TSP'] = 36638044]

SyntaxError: cannot assign to subscript here. Maybe you meant '==' instead of '='? (822543036.py, line 1)

In [10]:
dfOA.collect().to_pandas()

# ['TSP'].unique()

,Cycle_Desc,Eff_Gas_Day,Loc,Loc_Name,Loc_Zn,Flow_Ind_Desc,Loc_Purp_Desc,IT,All_Qty_Avail,Total_Design_Capacity,Operating_Capacity,Total_Scheduled_Quantity,Operationally_Available_Capacity,TSP
0,INTRDY_2025-11-16_0900,11-15-2025,00001,"Partroy (MORRIS,NJ)",None,Delivery,Delivery Location,N,Y,"18,022","18,022","5,000","13,022",006951446
1,INTRDY_2025-11-16_0900,11-15-2025,00002,"Waterbury (NEW HAVEN,CT)",None,Delivery,Delivery Location,N,Y,"208,442","216,006","13,109","202,897",006951446
2,INTRDY_2025-11-16_0900,11-15-2025,00003,North Haven (NEW HAVEN),None,Delivery,Delivery Location,N,Y,"342,081","342,081","62,074","280,007",006951446
3,INTRDY_2025-11-16_0900,11-15-2025,00004,"Rhode Island Energy -DEY ST. (PROVIDENCE,RI)",None,Delivery,Delivery Location,N,Y,"157,437","157,437","35,839","121,598",006951446
4,INTRDY_2025-11-16_0900,11-15-2025,00005,"Willimantic (NEW HAVEN, CT)",None,Delivery,Delivery Location,N,Y,"15,880","15,880","4,018","11,862",006951446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2799,INTRDYC_2025-11-16_2030,11-16-2025,45319,TENNESSEE GAS MAINLINE (TPGS1030B),None,Delivery,Delivery Location,N,Y,"712,396","712,396","19,001","693,395",791204600
2800,INTRDYC_2025-11-16_2030,11-16-2025,45321,ENTERPRISE TEXAS (VALERO) (TPGS1020B),None,Receipt,Receipt Location,Y,Y,"287,472","309,861","10,844","299,017",791204600
2801,INTRDYC_2025-11-16_2030,11-16-2025,45321,ENTERPRISE TEXAS (VALERO) (TPGS1020B),None,Delivery,Delivery Location,N,Y,"407,083","407,083","18,683","388,400",791204600
2802,INTRDYC_2025-11-16_2030,11-16-2025,45323,MATTERHORN PIPELINE,None,Delivery,Delivery Location,N,Y,"508,854","508,854",0,"508,854",791204600


In [22]:
selectedCols = ['Cycle_Desc', 'Eff_Gas_Day', 'Loc', 'Loc_Name', 'Loc_Zn', 'Flow_Ind_Desc', 'Loc_Purp_Desc', 'IT', 'All_Qty_Avail',
                'Total_Design_Capacity', 'Operating_Capacity', 'Total_Scheduled_Quantity', 'Operationally_Available_Capacity']


In [29]:
df = pl.scan_csv(Path('./downloads/enbridge/OA') / "*_OA_*.csv",
                    glob=True,
                    has_header=True,
                    )\
    .filter(
    ~functools.reduce(
        operator.and_,
        map(lambda col: pl.col(col).is_null(), selectedCols)
    ))


In [30]:
df=df.collect().to_pandas()

In [37]:
df[df['TSP'].isin(['612195573','946822749','36638044'])]['Loc_Name'].unique()

array(['PEPL MAUMEE', 'ANR FALLEN TIMBERS', 'OREGON CLEAN ENERGY PLANT',
       'Interstate Chemical', 'CLEVELAND CLIFFS', 'OLYMPUS (992003)',
       'VENICE GAS PLANT (992002)', 'VENICE GAS PLANT BYPASS (992010)',
       'KINETICA ENERGY EXPRESS (992009)',
       'HIGH POINT GAS TRANSMISSION (992006)', 'Ursa Platform MC 809-2',
       'MC 989 Vito', 'MARS PLATFORM (R-M0022 D-M0024)',
       'EARLY GROVE REC/INJ (59147)', 'SALTVILLE REC INTO P-25',
       'EARLY GROVE DEL/WD (59009)',
       'United Salt - Evaporator Fuel Gas - Smyth Co., VA',
       'SALTVILLE LLC STORAGE INJECTION', 'SALTVILLE STORAGE WITHDRAWAL',
       'SALTVILLE LLC STORAGE WITHDRAWAL', 'SALTVILLE STORAGE INJECTION'],
      dtype=object)

In [ ]:
len(df[~df.isnull().any(axis=1)])

In [ ]:
df[df[selectedCols].isnull().any(axis=1)]

In [ ]:
dfOA.join(pipesLF, on='TSP', how='outer')